# Import statements & utilities

In [1]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.2 MB/s eta 0:00:00


In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [3]:
!pip -q install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [4]:
import wandb

In [5]:
from urllib import request
import pandas as pd
from collections import Counter
import random
import numpy as np
import pandas as pd
import copy
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [6]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [7]:
from dont_patronize_me import DontPatronizeMe

In [8]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

In [9]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [10]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# Load data

In [11]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)


In [12]:
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [24]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [23]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
tedf1 = pd.DataFrame(rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [15]:
rows_list = tedf1.values.tolist()

# Shuffle the list using random.shuffle
random.shuffle(rows_list)

# Create a new DataFrame from the shuffled list
tedf1 = pd.DataFrame(rows_list, columns=tedf1.columns)
tedf1

,par_id,community,text,label
0,8681,refugee,Rescue teams are still present on the island t...,0
1,9795,disabled,Kenya 's Minister of Health Cheopa Mailu share...,0
2,8558,migrant,Since Sri Lanka is the only country in the who...,0
3,8512,hopeless,""""""" We felt hopeless when we saw there was no ...",0
4,8455,refugee,"""But HRW has called for """" unimpeded and uncon...",0
...,...,...,...,...
2089,8437,vulnerable,"New World Development ( NWD , 00017 . HK ) own...",0
2090,9950,homeless,"In June , several NGOs decided to reduce food ...",0
2091,9858,in-need,"In total , FAO 's latest food insecurity asses...",0
2092,10271,immigrant,There was no immediate claim of responsibility...,0


# Add augmentation

In [25]:
# before augmentation
trdf1['label'].value_counts().to_frame()

,label
0,7581
1,794


In [26]:
trdf1['label'].value_counts(normalize=True).to_frame()

,label
0,0.905194
1,0.094806


In [27]:
fig = px.pie(trdf1, names='label')
fig.update_layout(width=650, height=400 , template="none")
fig.show()


In [18]:
augmented_data = pd.read_csv("pcl_augmented_data.csv", index_col=0)

In [28]:
augmented_data

,community,text,label
0,poor-families,"The plan saw approximately 150,000 children fr...",1
1,homeless,Lunch of reconciliation for the homeless commu...,1
2,poor-families,The next immediate problem that arose was how ...,1
3,vulnerable,Much more important than the implications for ...,1
4,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...
789,poor-families,Quoting the fact that these children who died ...,1
790,disabled,Fernández was a well-known philanthropist who ...,1
791,homeless,"Moved by their difficult situation, the Comman...",1
792,women,She reiterated her ministry's commitment to im...,1


In [29]:
trdf1 = pd.concat([trdf1, augmented_data], ignore_index=True)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
9164,NaN,poor-families,Quoting the fact that these children who died ...,1
9165,NaN,disabled,Fernández was a well-known philanthropist who ...,1
9166,NaN,homeless,"Moved by their difficult situation, the Comman...",1
9167,NaN,women,She reiterated her ministry's commitment to im...,1


In [30]:
# after augmentation

trdf1['label'].value_counts().to_frame()

,label
0,7581
1,1588


In [31]:
trdf1['label'].value_counts(normalize=True).to_frame()

,label
0,0.826808
1,0.173192


In [32]:
fig = px.pie(trdf1, names='label')
fig.update_layout(width=650, height=400 , template="none")
fig.show()


# Load model

In [33]:
# Load model

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

# Create Datasets & DataLoaders

In [34]:
import datasets
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

In [35]:
# Split the dataset into train and test sets
train_df, val_df = train_test_split(trdf1, test_size=0.2, random_state=42)

train_dataset =  Dataset.from_dict({"data": train_df["text"], "label": train_df["label"]}).with_format("torch")
val_dataset =  Dataset.from_dict({"data": val_df["text"], "label": val_df["label"]}).with_format("torch")
test_dataset =  Dataset.from_dict({"data": tedf1["text"], "label": tedf1["label"]}).with_format("torch")

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

7335
1834
2094


In [36]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/7335 [00:00<?, ? examples/s]

Map:   0%|          | 0/1834 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

# Train Model

In [37]:
from sklearn.metrics import accuracy_score, f1_score

# Define evaluation metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')


    # return {"accuracy": accuracy, "f1": f1, "predictions": list(predictions)}
    return {"accuracy": accuracy, "f1": f1}

In [39]:
batch_size = 16

training_args = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="roberta-base-hate-best-augmented",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                  )


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [40]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.049600,0.290387,0.877863,0.622896
2,0.194100,0.243499,0.899128,0.715822
3,0.469400,0.244762,0.907306,0.724919
4,0.067400,0.256488,0.910033,0.729064
5,0.002900,0.302961,0.913850,0.738411
6,0.142500,0.324377,0.908397,0.739938
7,0.001600,0.357136,0.907852,0.741194
8,0.118800,0.361028,0.910033,0.740973
9,0.007500,0.372830,0.909487,0.746951
10,0.583500,0.373410,0.908397,0.743119


TrainOutput(global_step=4590, training_loss=0.18739079659711366, metrics={'train_runtime': 1882.9035, 'train_samples_per_second': 38.956, 'train_steps_per_second': 2.438, 'total_flos': 4905881377149960.0, 'train_loss': 0.18739079659711366, 'epoch': 10.0})

# Predicitons on val set

In [41]:
predictions = trainer.predict(tokenized_val_dataset).predictions
predictions = predictions.argmax(axis=1)
predictions

array([0, 0, 0, ..., 1, 0, 0])

In [42]:
Counter(predictions)

Counter({0: 1510, 1: 324})

In [43]:
labels2file([[k] for k in predictions], 'val.txt')

In [44]:
trainer.predict(tokenized_val_dataset).metrics

{'test_loss': 0.37341007590293884,
 'test_accuracy': 0.9083969465648855,
 'test_f1': 0.7431192660550459,
 'test_runtime': 12.8175,
 'test_samples_per_second': 143.086,
 'test_steps_per_second': 8.972}

In [45]:
results = trainer.evaluate(tokenized_val_dataset)

print(results)

{'eval_loss': 0.37341007590293884, 'eval_accuracy': 0.9083969465648855, 'eval_f1': 0.7431192660550459, 'eval_runtime': 12.8183, 'eval_samples_per_second': 143.076, 'eval_steps_per_second': 8.972, 'epoch': 10.0}


# Predictions on dev set (our test set)

In [46]:
predictions = trainer.predict(tokenized_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_loss': 0.3147519826889038, 'test_accuracy': 0.9226361031518625, 'test_f1': 0.5736842105263158, 'test_runtime': 14.7247, 'test_samples_per_second': 142.21, 'test_steps_per_second': 8.897}


array([0, 1, 0, ..., 0, 0, 0])

In [47]:
labels2file([[k] for k in predictions], 'dev.txt')

In [48]:
Counter(predictions)

Counter({0: 1913, 1: 181})

# Predictions on official test set

In [49]:
dpm.load_test()
test_df = dpm.test_set_df

In [50]:
test_df

,par_id,art_id,keyword,country,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to..."
1,t_1,@@16397324,women,pk,In most poor households with no education chil...
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population..."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C..."
...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina..."
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...


In [51]:
official_test_dataset =  Dataset.from_dict({"data": test_df["text"]}).with_format("torch")
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [52]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 28.3824, 'test_samples_per_second': 135.013, 'test_steps_per_second': 8.456}


array([0, 0, 0, ..., 0, 0, 0])

In [53]:
labels2file([[k] for k in predictions], 'test.txt')